In [8]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# Funciones para generar trayectorias
def brownian_motion(n_steps, speed, start_x, start_y):
    x = [start_x]
    y = [start_y]
    z = [0]  # Altura constante para mantener 2D
    for _ in range(1, n_steps):
        x.append(x[-1] + np.random.normal(0, speed))
        y.append(y[-1] + np.random.normal(0, speed))
        z.append(0)  # Mantener z constante
    return pd.DataFrame({'x': x, 'y': y, 'z': z})

def correlated_random_walk(n_steps, speed, start_x, start_y, cauchy_coeff):
    x = [start_x]
    y = [start_y]
    z = [0]
    for _ in range(1, n_steps):
        angle = np.random.standard_cauchy() * cauchy_coeff
        x.append(x[-1] + speed * np.cos(angle))
        y.append(y[-1] + speed * np.sin(angle))
        z.append(0)
    return pd.DataFrame({'x': x, 'y': y, 'z': z})

def levy_flight(n_steps, speed, start_x, start_y, cauchy_coeff, alpha):
    x = [start_x]
    y = [start_y]
    z = [0]
    for _ in range(1, n_steps):
        step_length = np.random.pareto(alpha) * speed
        angle = np.random.standard_cauchy() * cauchy_coeff
        x.append(x[-1] + step_length * np.cos(angle))
        y.append(y[-1] + step_length * np.sin(angle))
        z.append(0)
    return pd.DataFrame({'x': x, 'y': y, 'z': z})

# Crear la aplicación Dash
app = Dash(__name__)

# Layout de la aplicación
app.layout = html.Div(style={'display': 'flex'} , children=[
    # Panel de controles
    html.Div(style={'width': '30%', 'padding': '10px'}, children=[
        html.H1("Trajectory Simulation"),
        dcc.RadioItems(
            id='trajectory-type',
            options=[
                {'label': 'Brownian Motion', 'value': 'BM'},
                {'label': 'Correlated Random Walk', 'value': 'CRW'},
                {'label': 'Levy Flight', 'value': 'LF'}
            ],
            value='BM',
            labelStyle={'display': 'block'}  # Muestra las opciones en bloque
        ),
        html.Label("Number of Steps:"),
        dcc.Input(id='n-steps', type='number', value=100, min=1),
        html.Label("Speed:"),
        dcc.Input(id='speed', type='number', value=1.0, step=0.1),
        html.Label("Starting X Position:"),
        dcc.Input(id='start-x', type='number', value=0.0),
        html.Label("Starting Y Position:"),
        dcc.Input(id='start-y', type='number', value=0.0),
        html.Label("Cauchy Coefficient (for CRW and LF):"),
        dcc.Input(id='cauchy-coeff', type='number', value=1.0, style={'display': 'none'}),  # Oculto inicialmente
        html.Label("Alpha (for LF):"),
        dcc.Input(id='alpha', type='number', value=1.0, style={'display': 'none'}),  # Oculto inicialmente
    ]),
    
    # Gráfico de trayectoria
    html.Div(style={'width': '70%', 'padding': '10px'}, children=[
        dcc.Graph(id='trajectory-graph'),
    ]),
])

# Callback para mostrar los parámetros adicionales dependiendo del tipo de trayectoria
@app.callback(
    [Output('cauchy-coeff', 'style'),
     Output('alpha', 'style')],
    Input('trajectory-type', 'value')
)
def update_parameters(trajectory_type):
    if trajectory_type == 'CRW':
        return {'display': 'block'}, {'display': 'none'}
    elif trajectory_type == 'LF':
        return {'display': 'block'}, {'display': 'block'}
    else:
        return {'display': 'none'}, {'display': 'none'}

# Callback para actualizar la gráfica de la trayectoria
@app.callback(
    Output('trajectory-graph', 'figure'),
    Input('trajectory-type', 'value'),
    Input('n-steps', 'value'),
    Input('speed', 'value'),
    Input('start-x', 'value'),
    Input('start-y', 'value'),
    Input('cauchy-coeff', 'value'),  # Ya no hay default
    Input('alpha', 'value')  # Ya no hay default
)
def update_trajectory(trajectory_type, n_steps, speed, start_x, start_y, cauchy_coeff, alpha):
    if trajectory_type == 'BM':
        trajectory = brownian_motion(n_steps, speed, start_x, start_y)
    elif trajectory_type == 'CRW':
        trajectory = correlated_random_walk(n_steps, speed, start_x, start_y, cauchy_coeff)
    elif trajectory_type == 'LF':
        trajectory = levy_flight(n_steps, speed, start_x, start_y, cauchy_coeff, alpha)

    # Crear gráfico 3D
    fig = go.Figure(data=[go.Scatter3d(
        x=trajectory['x'],
        y=trajectory['y'],
        z=trajectory['z'],  # Z puede ser un valor que desees representar
        mode='lines'
    )])
    
    fig.update_layout(title=f"{trajectory_type} Trajectory", scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ))
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
